In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#set directory for importinh my_tools,py
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import file_dic, plot_dic
from datetime import datetime
import glob
from mpl_toolkits.basemap import Basemap, addcyclic
%matplotlib qt

In [2]:
#Set path. Change accordingly
path_file = '/media/onno/Algemeen/Thesis/'
path =  '/media/onno/Algemeen/Thesis/GFS_climatology/'
#open dataset
gfs = xr.open_dataset(path_file+file_gfs,decode_times=False)
gfs['time']=my_tools.convert_date_gefs_r(gfs.time.values)
#only select lead times at 24 h intervals
gfs = gfs.sel(lead=gfs.lead.values[[bool(i) for i in gfs.lead%24==0]])

In [5]:
#select variables to calculate mean and loop over them
variablez = ['phasespeed','u_wind']
for variable in variablez:
    file_gfs = file_dic[variable]['GFS']
    #open dataset
    gfs = xr.open_dataset(path_file+file_gfs,decode_times=False)
    gfs['time']=my_tools.convert_date_gefs_r(gfs.time.values)
    #select only gfs values at 24 hour intervals
    gfs = gfs.sel(lead=gfs.lead.values[[bool(i) for i in gfs.lead%24==0]])
    #loop over all lead days
    for i in range(11):
        gfs_lead = gfs.sel(lead=i*24).squeeze()
        #calculate climatology 
        gfs_lead_mean = gfs_lead.mean(dim='time')
        #save climatology to netcf
        gfs_lead_mean.to_netcdf(path+'GFS_{}_clim_day_{:02d}'.format(variable,i))
    

In [ ]:
variablez = ['envelope','phasespeed','u_wind']
fig,axz = plt.subplots(2,5,figsize=(20,10))
for variable in variablez:
    for i in range(10):
        ax = axz.flat[i]
        gfs_lead_mean = xr.open_dataset(path+'/GFS_{}_clim_day_{:02d}.nc'.format(variable,i+1),decode_times=False)
        #Create cyclic point so data is continious to sue for contourf

        gfs_lead_mean_cyclic, lon_cyclic = addcyclic(gfs_lead_mean[plot_dic[variable]['data_variable']].values, gfs_lead_mean.lon)
        lon,lat = np.meshgrid(lon_cyclic,gfs_lead_mean.lat)

        m = Basemap(projection='ortho',lon_0=0,lat_0=90,resolution='c',ax=ax)
        im = m.contourf(lon,lat,gfs_lead_mean_cyclic,latlon='true', extend='both', cmap=plt.cm.get_cmap('viridis'),
                   vmin=0,vmax=12,levels=np.linspace(0,12,49))
        m.drawcoastlines()
        m.drawparallels((-80,-60,-40,-20,0,20,40,60,80))
        m.drawmeridians((0,60,120,180,240,300),labels=[0,0,1,1])
    fig.subplots_adjust(top=0.85,bottom=0.05,left=0,
                        right=0.8,hspace=0.3,wspace=0.0)
    cbax = fig.add_axes([0.85, 0.2, 0.025, 0.6])
    fig.colorbar(im, cax=cbax)
    cbax.set_ylabel(plot_dic[variable]['label'], rotation=90)
    fig.suptitle('GEFS {} Climatology'.format(plot_dic[variable]['title']))
    fig.subplots_adjust(right=0.84)
    sys.exit()
    fig.savefig(path+'GEFS_{}_climatology_mars.png'.format(variable))
    plt.close(fig)

In [22]:
#same procedure as the annual means but now done for each season
seasons = {'MAM':[3,4,5],
          'JJA':[6,7,8],
          'SON':[9,10,11],
          'DJF':[12,1,2]}
variablez=['envelope','phasespeed','u_wind']
for variable in variablez:
    file_gfs = file_dic[variable]['GFS']
    #open dataset
    gfs = xr.open_dataset(path_file+file_gfs,decode_times=False)
    gfs['time']=my_tools.convert_date_gefs_r(gfs.time.values)
    gfs = gfs.sel(lead=gfs.lead.values[[bool(i) for i in gfs.lead%24==0]])
    for key in seasons:
        for i in range(11):
            gfs_lead = gfs.sel(lead=i*24)
            idxz = np.isin(gfs_lead.time.dt.month,seasons[key])
            assert(gfs_lead.time[idxz][0].dt.month==seasons[key][0])
            assert(gfs_lead.time[idxz][-1].dt.month==seasons[key][-1])
            gfs_lead_season_mean=gfs_lead[plot_dic[variable]['data_variable']][idxz].mean(dim='time')
            gfs_lead_season_mean.to_netcdf(path+'/GFS_{}_clim_{}_day_{:02d}.nc'.format(variable,key,i))
    